In [1]:
from parse_text import spacy_txt
import pandas as pd
from snapy import MinHash, LSH
import time
import random
import re
import copy

In [8]:
word_id = 'doc11.txt_sentence_14_noun_32'
#doc_index=int(re.sub('(?<=sentence).+','',word_id).replace('sentence',''))
#index = re.sub('.+(?=sentence)','',word_id).replace('sentence','')
#word_index = int(re.sub('.+(?=_)','',index).replace('_',''))
#sent_index= int(re.sub('(?<=_).+','',index).replace('_',''))

doc_index = re.findall('.+(?=_sentence)',word_id)[0]
sent_index= int(re.findall('(?<=sentence_).+(?=_noun)',word_id)[0])
noun_index=int(re.findall('(?<=noun_).+',word_id)[0])
noun_index

32

In [ ]:
path = "DEA"
s = spacy_txt()
doc, e, n= s.read_file(path)

In [ ]:
mesh_fp = '/teams/DSC180A_FA20_A00/a07opioidoverdoseprevalence/data/MESH.csv'
rxnorm_fp = '/teams/DSC180A_FA20_A00/a07opioidoverdoseprevalence/data/RXNORM.csv'
rx=pd.read_csv(rxnorm_fp,usecols=['Preferred Label','Semantic type UMLS property'])
mesh=pd.read_csv(mesh_fp,usecols=['Preferred Label','Semantic type UMLS property'])
term = pd.concat([rx,mesh])
term['Preferred Label'] = term['Preferred Label'].str.lower()
ontology = dict(zip(term['Preferred Label'],term['Semantic type UMLS property']))
terms = list(ontology.keys())
#terms = terms[:50000]

In [ ]:
terms = sorted([i.strip('(+)-').strip('(),.-{').strip().replace("'",'').replace("}",'') for i in terms])
dic={}
for i in terms:
    c = i[0]
    if c in dic:
        dic[c]+=1
    else:
        dic[c]=1
dic

In [ ]:
def make_content_text(doc,doc_id):
    #Initialize a dictionary
    content = dict()       

    #Add noun to dictionary with its location as key
    for sent_id in doc.sentences:
        sent=doc.sentences[sent_id]
        for n_id,noun in enumerate(sent.noun):
            key=str(doc_id)+'sentence'+str(sent_id)+'_'+str(n_id)
            noun = noun.lower()
            if len(noun) > 3:
                content[key] = noun
               
    return content

In [ ]:
def make_content_search(search_terms):
    #Initialize a dictionary
    content = dict()
    
    #Add search terms to dictionary with numbered key
    for i in range(len(search_terms)):
        if len(search_terms[i]) > 3:
            content[i] = search_terms[i].strip().lower()    
    return content

In [ ]:
content = make_content_search(terms)
for doc_id,document in enumerate(doc):
    content.update(make_content_text(document,doc_id))

In [ ]:
def create_lsh(content, n_permutations, n_gram):
    labels = content.keys()
    values = content.values()
    #Create MinHash object
    minhash = MinHash(values, n_gram=n_gram, permutations=n_permutations, hash_bits=64, seed=3)
    #Create LSH model
    lsh = LSH(minhash, labels, no_of_bands=5)
    return lsh

In [ ]:
n_permutations= 50,
n_gram= 3
%time lsh = create_lsh(content,n_permutations,n_gram)

In [ ]:
edge_list = lsh.edge_list(min_jaccard=0.2,jaccard_weighted=True)

In [ ]:
start = time.time()
similar_ls = []
for i in edge_list:
    if type(i[1])==int and type(i[0])==str:
        similar_ls.append(i)
        print(edge_list.index(i))
    elif (type(i[0])==int and type(i[1])==str):
        similar_ls.append(i)
        print(edge_list.index(i))
    elif type(i[0])==int and type(i[1])==int:
        break
end = time.time()
end-start

In [ ]:
similar_ls

In [ ]:
def update_entity_type(docs,similar_ls,terms,ontology):
    for i in similar_ls:
        word_id = i[0]
        term_id = i[1]
        score = i[2]
        #get the entity 
        word = content[word_id]
        #get the sentence and word index of the entity
        doc_index=int(re.sub('(?<=sentence).+','',word_id).replace('sentence',''))
        index = re.sub('.+(?=sentence)','',word_id).replace('sentence','')
        word_index = int(re.sub('.+(?=_)','',index).replace('_',''))
        sent_index= int(re.sub('(?<=_).+','',index).replace('_',''))

        types = []
        scores = []
        # get ontology type for entity
        onto = ontology[terms[term_id]]
        types.append(onto)
        scores.append(score)
        
        #check if the noun is already in ner list
        in_ner =False
        ner_score =1
        doc = docs[doc_index]
        sent = doc.sentences[sent_index]
        #loop through list of entity dictionary for the sentence
        for entity in sent.ner:
            current = list(entity.keys())[0]
            if word==current.lower():
                types.append(entity[current])
                scores.append(ner_score)
                entity[current] = [types,scores]
                in_ner = True
                print(doc_index,sent_index)
                
        if not in_ner:
            #store type of entity to its location in the list
            sent.ner.append({word:[types,scores]})
            print(doc_index,sent_index)

In [ ]:
update_entity_type(doc,similar_ls,terms,ontology)

In [ ]:
doc.sentence.ner